In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from torch import nn, optim
import torch
from sklearn import preprocessing
from IPython import display
from sklearn.model_selection import train_test_split

In [2]:
#this should be done to test data
test = pd.read_csv("test.csv")
# passengerid_test_values = np.zeros((418 , 1) , dtype=np.int)
# passengerid_test_values[:][0] = test['PassengerId'].values
train = pd.read_csv("train.csv")
print("Dimensions of train: {}".format(train.shape))
print("Dimensions of test: {}".format(test.shape))

Dimensions of train: (891, 12)
Dimensions of test: (418, 11)


In [3]:
all_X = train[:]
all_y = train['Survived']

test_passid = test['PassengerId']

train_X, eval_X, train_Y, eval_Y = train_test_split(
    all_X, all_y, test_size=0.20,random_state=0)

In [4]:
#this should be done to test data
def process_age(df,cut_points,label_names):
    df["Age"] = df["Age"].fillna(-0.5)
    df["Age_categories"] = pd.cut(df["Age"],cut_points,labels=label_names)
    return df

cut_points = [-1,0,5,12,18,35,60,100]
label_names = ["Missing","Infant","Child","Teenager","Young Adult","Adult","Senior"]
train_X = process_age(train_X,cut_points,label_names)
eval_X = process_age(eval_X,cut_points,label_names)
test = process_age(test,cut_points,label_names)


# pivot = train.pivot_table(index="Age_categories",values='Survived')
# pivot.plot.bar()
# plt.show()

C:\Users\Mukhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Mukhtar\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [5]:
#this should be done to test data
train_X = train_X[["Pclass","Sex","Age_categories"]]
eval_X = eval_X[["Pclass","Sex","Age_categories"]]
test = test[["Pclass","Sex","Age_categories"]]
le = preprocessing.LabelEncoder()
train_X = train_X[train_X.columns[:]].apply(le.fit_transform)
eval_X = eval_X[eval_X.columns[:]].apply(le.fit_transform)
test = test[test.columns[:]].apply(le.fit_transform)


train_X = torch.tensor(train_X[:].values)
train_Y = torch.tensor(train_Y[:].values)
train_X = train_X.type(torch.FloatTensor)
train_Y = train_Y.squeeze_()

test = torch.tensor(test[:].values)
test = test.type(torch.FloatTensor)
test_passid_values = torch.tensor(test_passid[:].values)

eval_X = torch.tensor(eval_X[:].values)
eval_Y = torch.tensor(eval_Y[:].values)
eval_X = eval_X.type(torch.FloatTensor)
eval_Y = eval_Y.squeeze_()

In [6]:
# X_test = test[["Pclass","Sex","Age_categories"]]
# Y_test = test[["Survived"]]
# le = preprocessing.LabelEncoder()
# X_test = X_test[X_test.columns[:]].apply(le.fit_transform)

# X_test = torch.tensor(X_train[:].values)
# Y_test = torch.tensor(Y_train[:].values)
# X_test = X_test.type(torch.FloatTensor)
# Y_test = Y_test.squeeze_()

In [7]:
print("train_X:", tuple(train_X.size()))
print("train_Y:", tuple(train_Y.size()))
print("eval_X:", tuple(eval_X.size()))
print("eval_Y:", tuple(eval_Y.size()))
print("test:", tuple(test.size()))

train_X: (712, 3)
train_Y: (712,)
eval_X: (179, 3)
eval_Y: (179,)
test: (418, 3)


In [8]:
# for t in range(X_train.size(0)):   
#     print(train_X.data[t])

In [9]:
#model difinition simple DL calssifier
model = torch.nn.Sequential(
    nn.Linear(3, 100),
    nn.ReLU(),
    nn.Linear(100, 2)
)

learning_rate = 1e-3
lambda_l2 = 1e-5

criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) # built-in L2

In [10]:
for Epoch in range(100000):
    y_pred = model(train_X);
    
    loss = criterion(y_pred,train_Y);
    
    score, predicted = torch.max(y_pred, 1)
    acc = (train_Y == predicted).sum().float() / len(train_Y)
    print("[EPOCH]: %i, [LOSS]: %.6f, [ACCURACY]: %.3f" % (Epoch, loss.item(), acc))
    display.clear_output(wait=True)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

[EPOCH]: 99999, [LOSS]: 0.436228, [ACCURACY]: 0.805


In [11]:
#evalution
y_predict = model(eval_X);
    
eval_loss = criterion(y_predict,eval_Y);
    
eval_score, eval_predicted = torch.max(y_predict, 1)
eval_acc = (eval_Y == eval_predicted).sum().float() / len(eval_Y)
print("[LOSS]: %.6f, [ACCURACY]: %.3f" % (eval_loss.item(), eval_acc))
# display.clear_output(wait=True)

[LOSS]: 0.386985, [ACCURACY]: 0.821


In [12]:
y_test_predict = model(test);
        
test_score, test_predicted = torch.max(y_test_predict, 1)

In [13]:
print("test_passid_values:", tuple(test_passid_values.size()))
print("test_predicted:", tuple(test_predicted.size()))

test_passid_values: (418,)
test_predicted: (418,)


In [14]:
test_predict_value = np.empty((418, 1) , dtype=np.int)
test_passid = np.empty((418, 1) , dtype=np.int)

for t in range(test_predicted.size(0)):   
#     print(test_predicted.data[t].numpy())
    test_predict_value[t][0] =  test_predicted.data[t].numpy()
    test_passid[t][0] =  test_passid_values.data[t].numpy()
#     print(test_passid[t][0])

# print("test_predict_value:", tuple(test_predict_value.size()))
# print("test_passid:", tuple(test_passid.size()))

In [15]:
submission_df = {"PassengerId":test_passid[:,0],
                 "Survived":test_predict_value[:,0]}
submission = pd.DataFrame(submission_df)

In [16]:
submission.to_csv("submission.csv" , index=False)